In [ ]:
import tensorflow as tf
print(tf.__version__)

In [4]:
!git clone https://github.com/duressa-feyissa/classifier-to-distinguish-between-tomato-leaves.git

# Change directory to the dataset folder
%cd classifier-to-distinguish-between-tomato-leaves

Cloning into 'classifier-to-distinguish-between-tomato-leaves'...
remote: Enumerating objects: 23139, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 23139 (delta 0), reused 3 (delta 0), pack-reused 23136
Receiving objects: 100% (23139/23139), 310.11 MiB | 22.27 MiB/s, done.
Updating files: 100% (23142/23142), done.
/content/classifier-to-distinguish-between-tomato-leaves/classifier-to-distinguish-between-tomato-leaves/classifier-to-distinguish-between-tomato-leaves


In [5]:
# List files to verify
!ls

dataset  README.md


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [7]:
train_data_dir = './dataset/train'
validation_data_dir = './dataset/validation'
img_width, img_height = 150, 150
batch_size = 32
epochs = 30

In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [9]:
validation_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 20708 images belonging to 2 classes.


In [11]:
validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary'
)

Found 2433 images belonging to 2 classes.


In [12]:
model = Sequential()

In [13]:
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [14]:
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [15]:
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [16]:
model.add(Flatten())

In [17]:
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

In [18]:
model.add(Dense(1, activation='sigmoid'))

In [19]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [20]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_tomato_leaf_model.h5', monitor='val_loss', save_best_only=True)

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[early_stopping, checkpoint]
)

Epoch 1/30
647/647 [==============================] - 892s 1s/step - loss: 0.2534 - accuracy: 0.8976 - val_loss: 0.0616 - val_accuracy: 0.9799


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/30
647/647 [==============================] - 838s 1s/step - loss: 0.1296 - accuracy: 0.9567 - val_loss: 0.1097 - val_accuracy: 0.9593
Epoch 3/30
647/647 [==============================] - 816s 1s/step - loss: 0.1014 - accuracy: 0.9685 - val_loss: 0.0412 - val_accuracy: 0.9840
Epoch 4/30
647/647 [==============================] - 795s 1s/step - loss: 0.0812 - accuracy: 0.9722 - val_loss: 0.0649 - val_accuracy: 0.9749
Epoch 5/30
647/647 [==============================] - 794s 1s/step - loss: 0.0649 - accuracy: 0.9809 - val_loss: 0.0197 - val_accuracy: 0.9942
Epoch 6/30
647/647 [==============================] - 774s 1s/step - loss: 0.0568 - accuracy: 0.9826 - val_loss: 0.0457 - val_accuracy: 0.9848
Epoch 7/30
315/647 [=============>................] - ETA: 6:19 - loss: 0.0374 - accuracy: 0.9890

In [ ]:
model.save('model.keras')

NameError: name 'model' is not defined

In [ ]:
Y_pred = model.predict(validation_generator)
y_pred = np.where(Y_pred > 0.5, 1, 0).flatten()  # Convert probabilities to binary outcomes
y_true = validation_generator.classes

77/77 [==============================] - 20s 261ms/step


In [ ]:
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix')
print(conf_matrix)

Confusion Matrix
[[  13  169]
 [ 172 2079]]


In [ ]:
class_report = classification_report(y_true, y_pred, target_names=['Non-Tomato Leaf', 'Tomato Leaf'])
print('Classification Report')
print(class_report)

Classification Report
                 precision    recall  f1-score   support

Non-Tomato Leaf       0.07      0.07      0.07       182
    Tomato Leaf       0.92      0.92      0.92      2251

       accuracy                           0.86      2433
      macro avg       0.50      0.50      0.50      2433
   weighted avg       0.86      0.86      0.86      2433



In [ ]:
roc_auc = roc_auc_score(y_true, Y_pred)
print('ROC-AUC Score')
print(roc_auc)

ROC-AUC Score
0.4903718005672692
ROC-AUC Score
0.4903718005672692


In [ ]:
# Function to preprocess input image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # rescale to 0-1
    return img_array

In [ ]:
# Function to predict if the image is a tomato leaf
def predict_image(img_path):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)
    if prediction[0] > 0.5:
        print("The image is a tomato leaf")
    else:
        print("The image is not a tomato leaf")

In [ ]:
predict_image('a.jpg')

1/1 [==============================] - 0s 31ms/step
The image is a tomato leaf
